In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.SGD_classifier

In [4]:
clients, text_x, test_y = utils.set_data(True)
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.001)
    client.prep_data()
    client.train_model()
    print(client.y.mean())
    y_hat = client.model.predict(text_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
0.36208
0.3635
0.36181
0.36647
0.36462


In [5]:
f1_before

[0.9562169016185574,
 0.9557097038860297,
 0.9558433033982756,
 0.9556656125067129,
 0.9563169037182813]

In [6]:
selected_model = Supported_modles.SGD_classifier

In [7]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,0.00001,None)

In [8]:
fedavg = Fedavg("global", 0.00001)
fedavg.init_global_model(selected_model, None,78)

[Errno 98] Address already in use
Waitiing for a Connection..


In [13]:
number_of_rounds=50
batch_size = 0.2
epochs = 10
prep = StandardScaler() 


for _ in range(number_of_rounds):
    print(_,end=" ")

    applicable_clients = random.sample((clients), random.randint(2, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [14]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = client.model.predict(text_x)
    f1_fedavg.append(f1_score(test_y,y_hat,average="binary"))
print(f1_fedavg)

[0.9755885960451133, 0.9642601545269983, 0.9769707048360036, 0.9758556725706079, 0.9715094673509114]


In [15]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.01937169442655584
Difference 0.008550450640968599
Difference 0.021127401437727955
Difference 0.02019006006389501
Difference 0.015192563632630107
